# SageMaker Model Monitor - Visualizing monitoring results


The prebuilt container from Amazon SageMaker computes a variety of statistics and evaluates constraints out of the box. This notebook demonstrates how you can visualize the results. You can get the ProcessingJob ARN from the executions behind a MonitoringSchedule and use it in this notebook to create a visualization.

## Contents
    [Get started with dependencies](#Get-started-with-dependencies)
    [Get utilities for rendering](#Get-utilities-for-rendering)
    [Get execution and baseline details from processing job ARN](#Get-execution-and-baseline-details-from-processing-job-ARN)
    [Overview table](#Overview-table)
    [Visualize distributions](#Visualize-distributions)

## Get started with dependencies

To begin setting up your visualizations, import some Python libraries.

In [ ]:
from IPython.display import HTML, display
import json
import os
import boto3

import sagemaker
from sagemaker import session
from sagemaker.model_monitor import MonitoringExecution
from sagemaker.s3 import S3Downloader

## Get utilities for rendering

Import the functions for plotting and rendering distribution statistics or constraint violations. They're implemented in a `utils` file.

In [ ]:
!wget https://raw.githubusercontent.com/awslabs/amazon-sagemaker-examples/master/sagemaker_model_monitor/visualization/utils.py

import utils as mu

## Get execution and baseline details from processing job ARN

Enter the ProcessingJob ARN for an execution of a MonitoringSchedule below to get the result files associated with that execution.

In [ ]:
processing_job_arn = "FILL-IN-PROCESSING-JOB-ARN" 

In [ ]:
execution = MonitoringExecution.from_processing_arn(sagemaker_session=session.Session(), processing_job_arn=processing_job_arn)
exec_inputs = {inp['InputName']: inp for inp in execution.describe()['ProcessingInputs']}
exec_results = execution.output.destination

In [ ]:
baseline_statistics_filepath = exec_inputs['baseline']['S3Input']['S3Uri'] if 'baseline' in exec_inputs else None
execution_statistics_filepath = os.path.join(exec_results, 'statistics.json')
violations_filepath = os.path.join(exec_results, 'constraint_violations.json')

baseline_statistics = json.loads(S3Downloader.read_file(baseline_statistics_filepath)) if baseline_statistics_filepath is not None else None
execution_statistics = json.loads(S3Downloader.read_file(execution_statistics_filepath))
violations = json.loads(S3Downloader.read_file(violations_filepath))['violations']

## Overview table

The code below shows the violations and constraint checks across all features in a simple table.

In [ ]:
mu.show_violation_df(baseline_statistics=baseline_statistics, latest_statistics=execution_statistics, violations=violations)

## Visualize distributions

This section visualizes the distribution and renders the distribution statistics for all features.

In [ ]:
features = mu.get_features(execution_statistics)
feature_baselines = mu.get_features(baseline_statistics)

In [ ]:
mu.show_distributions(features)

### Execution Stats vs Baseline

In [ ]:
mu.show_distributions(features, feature_baselines)